In [1]:
# !kaggle competitions download -c ds2-tree-ensembles

In [2]:
# !unzip test_features.csv.zip
# !unzip train_features.csv.zip
# !unzip train_labels.csv.zip

In [1]:
import pandas as pd
import numpy as np
import eli5
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
pd.options.display.max_columns=(999)
pd.options.display.max_rows=(999)

In [2]:
train = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')
sample_submission = pd.read_csv('sample_submission.csv')
print(train.shape, train_labels.shape, test.shape)

(1309457, 103) (1309457, 2) (26724, 103)


In [3]:
train.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,1246716,NaN,22000.0,22000.0,36 months,5.32%,662.53,A,A1,Transition Coordinator,10+ years,MORTGAGE,126000.0,NaN,NaN,home_improvement,Home improvement,604xx,IL,28.99,0.0,Oct-2000,0.0,43.0,NaN,12.0,0.0,9371.0,51%,18.0,w,0.0,29.0,Individual,NaN,NaN,0.0,0.0,367985.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18351.0,NaN,NaN,NaN,2.0,30665.0,3250.0,69.0,0.0,0.0,145.0,182.0,5.0,2.0,2.0,57.0,NaN,18.0,NaN,2.0,3.0,7.0,4.0,4.0,7.0,7.0,9.0,7.0,12.0,NaN,0.0,0.0,2.0,89.0,0.0,0.0,0.0,451899.0,198749.0,9750.0,210298.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,1678744,NaN,10000.0,10000.0,36 months,5.32%,301.15,A,A1,Director HQ Controlling,4 years,MORTGAGE,120000.0,NaN,NaN,debt_consolidation,Debt consolidation,019xx,MA,10.69,1.0,Jan-2002,0.0,13.0,NaN,13.0,0.0,9145.0,35%,46.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,40945.0,0.0,9.0,0.0,4.0,13.0,31800.0,66.0,0.0,0.0,5548.0,55.0,26100.0,0.0,1.0,0.0,4.0,3150.0,16355.0,35.9,0.0,0.0,168.0,116.0,31.0,13.0,0.0,31.0,NaN,20.0,13.0,0.0,2.0,2.0,3.0,4.0,41.0,4.0,5.0,2.0,13.0,0.0,0.0,0.0,0.0,97.8,0.0,0.0,0.0,74267.0,40945.0,25500.0,48167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,1149018,NaN,7000.0,7000.0,36 months,6.97%,216.05,A,A3,NaN,NaN,RENT,80000.0,NaN,NaN,major_purchase,Major purchase,653xx,MO,29.39,0.0,Jan-2002,2.0,NaN,NaN,13.0,0.0,3703.0,10.4%,20.0,f,0.0,NaN,Individual,NaN,NaN,0.0,0.0,78269.0,1.0,5.0,2.0,3.0,7.0,74566.0,66.0,2.0,3.0,546.0,45.0,35500.0,2.0,0.0,5.0,6.0,6021.0,17930.0,6.1,0.0,0.0,170.0,44.0,4.0,4.0,0.0,4.0,NaN,1.0,NaN,0.0,3.0,6.0,4.0,5.0,10.0,8.0,10.0,6.0,13.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,131514.0,78269.0,19100.0,96014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,1349740,NaN,1400.0,1400.0,36 months,13.67%,47.63,C,C4,Administrative Assistant,5 years,RENT,49000.0,NaN,NaN,medical,Medical expenses,770xx,TX,25.07,0.0,Aug-2004,0.0,46.0,NaN,12.0,0.0,19979.0,79.6%,18.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,38644.0,1.0,1.0,1.0,1.0,5.0,18665.0,96.0,0.0,2.0,7612.0,87.0,25100.0,0.0,0.0,0.0,3.0,3220.0,1411.0,93.2,0.0,0.0,111.0,136.0,16.0,5.0,0.0,16.0,NaN,16.0,46.0,0.0,7.0,8.0,7.0,8.0,2.0,11.0,16.0,8.0,12.0,0.0,0.0,0.0,1.0,94.4,100.0,0.0,0.0,44615.0,38644.0,20800.0,19515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,1234365,NaN,19200.0,19200.0,36 months,8.18%,603.26,B,B1,Plant Electrical Engineer,< 1 year,RENT,150000.0,NaN,NaN,credit_card,Credit card refinancing,971xx,OR,16.18,0.0,Oct-1997,3.0,NaN,78.0,8.0,1.0,27000.0,56%,17.0,f,0.0,31

In [4]:
test.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,2264704,NaN,20000.0,20000.0,36 months,8.18%,628.39,B,B1,Superintendent,2 years,MORTGAGE,64000.00,NaN,NaN,debt_consolidation,Debt consolidation,320xx,FL,12.45,0.0,Dec-2002,0.0,28.0,NaN,9.0,0.0,22607.0,82.2%,12.0,f,0.0,NaN,Individual,NaN,NaN,0.0,50.0,36978.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27500.0,NaN,NaN,NaN,0.0,4109.0,2893.0,88.7,0.0,0.0,50.0,147.0,56.0,31.0,1.0,88.0,28.0,NaN,28.0,0.0,3.0,3.0,4.0,6.0,4.0,5.0,7.0,3.0,9.0,0.0,0.0,0.0,0.0,91.7,75.0,0.0,0.0,44000.0,36978.0,25500.0,16500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,1896343,NaN,4000.0,4000.0,36 months,13.99%,136.70,C,C1,Seats Inc,5 years,RENT,32000.00,NaN,NaN,debt_consolidation,Bills be gone,539xx,WI,16.20,0.0,Sep-2006,0.0,32.0,NaN,8.0,0.0,5240.0,77.1%,12.0,f,0.0,NaN,Individual,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1352.0,79.2,0.0,0.0,NaN,NaN,NaN,NaN,0.0,7.0,NaN,7.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,0.0,0.0,NaN,15701.0,6500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,2077525,NaN,10000.0,10000.0,36 months,10.99%,327.34,B,B2,Registered Nurse,6 years,MORTGAGE,89000.00,NaN,NaN,debt_consolidation,Debt consolidation,770xx,TX,18.04,2.0,Nov-2005,0.0,5.0,NaN,12.0,0.0,10675.0,87.6%,21.0,w,0.0,5.0,Individual,NaN,NaN,0.0,0.0,42555.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12500.0,NaN,NaN,NaN,3.0,3869.0,418.0,90.5,0.0,0.0,100.0,95.0,8.0,8.0,0.0,40.0,5.0,NaN,5.0,1.0,3.0,8.0,3.0,3.0,9.0,9.0,11.0,8.0,12.0,0.0,0.0,2.0,1.0,100.0,66.7,0.0,0.0,58690.0,42555.0,1400.0,46190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,1846827,NaN,12000.0,12000.0,60 months,14.31%,281.16,C,C4,Personal Banker,2 years,RENT,35360.00,NaN,NaN,debt_consolidation,Debt consolidation,805xx,CO,30.96,0.0,May-2003,0.0,48.0,NaN,7.0,0.0,6444.0,37.9%,16.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,86687.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17000.0,NaN,NaN,NaN,2.0,12384.0,2093.0,73.8,0.0,0.0,126.0,140.0,9.0,9.0,3.0,9.0,48.0,NaN,48.0,0.0,3.0,4.0,4.0,6.0,3.0,6.0,10.0,4.0,7.0,0.0,0.0,0.0,2.0,93.7,75.0,0.0,0.0,117300.0,86687.0,8000.0,100300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,1210251,NaN,2400.0,2400.0,36 months,6.72%,73.80,A,A3,Respiratory Therapist,5 years,OWN,60042.55,NaN,NaN,other,Other,473xx,IN,4.20,0.0,Apr-2000,0.0,NaN,NaN,14.0,0.0,4355.0,7.6%,31.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,5998.0,0.0,1.0,1.0,1.0,8.0,1643.0,82.0,1.0,1.0,1302.0,10.

In [5]:
train_labels.head()

,id,charged_off
0,1246716,0
1,1678744,0
2,1149018,1
3,1349740,0
4,1234365,0


In [6]:
train.describe()

,id,member_id,loan_amnt,funded_amnt,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
count,1.309457e+06,0.0,1.309457e+06,1.309457e+06,1.309457e+06,1.309457e+06,0.0,1.309130e+06,1.309457e+06,1.309456e+06,649098.000000,222755.000000,1.309457e+06,1.309457e+06,1.309457e+06,1.309457e+06,1.309404e+06,344343.000000,2.410300e+04,24100.000000,1.309457e+06,1.243219e+06,1.243219e+06,517932.000000,517933.000000,517933.000000,517933.000000,504297.000000,5.179330e+05,448281.000000,517933.000000,517933.000000,517933.000000,517882.000000,1.243219e+06,517933.000000,517932.000000,517932.000000,1.263098e+06,1.243198e+06,1.249649e+06,1.248904e+06,1.309404e+06,1.309457e+06,1.206228e+06,1.243218e+06,1.243218e+06,1.243219e+06,1.263098e+06,1.250551e+06,310720.000000,1.139613e+06,438251.000000,1.243219e+06,1.243219e+06,1.243219e+06,1.254709e+06,1.243219e+06,1.243219e+06,1.243219e+06,1.243218e+06,1.243219e+06,1.254709e+06,1.194612e+06,1.243219e+06,1.243219e+06,1.243219e+06,1.243068e+06,1.249244e+06,1.308775e+06,1.309420e+06,1.243219e+06,1.263098e+06,1.263098e+06,1.243219e+06,17084.000000,17085.000000,17085.000000,17085.000000,16776.000000,17085.000000,17085.000000,17085.000000,17085.000000,6108.000000
mean,1.668095e+06,NaN,1.441781e+04,1.440935e+04,4.380447e+02,7.621709e+04,NaN,1.828017e+01,3.184877e-01,6.558029e-01,34.287963,70.562290,1.159434e+01,2.157284e-01,1.625817e+04,2.498974e+01,1.712764e-02,43.683467,1.168734e+05,18.892332,5.083787e-03,2.499473e+02,1.410480e+05,1.050165,2.795549,0.775031,1.741262,19.615229,3.585163e+04,71.427625,1.421497,3.006729,5539.675782,58.267891,3.275146e+04,1.089237,1.609317,2.312340,4.689423e+00,1.347874e+04,1.014386e+04,6.004603e+01,9.066720e-03,1.497897e+01,1.258360e+02,1.815475e+02,1.311462e+01,7.858128e+00,1.672125e+00,2.382611e+01,39.632618,6.713107e+00,35.778196,5.097219e-01,3.644709e+00,5.647964e+00,4.733463e+00,8.106414e+00,8.567947e+00,8.279736e+00,1.461920e+01,5.597126e+00,1.164045e+01,8.262097e-04,3.455546e-03,8.910739e-02,2.176871e+00,9.416167e+01,4.527371e+01,1.344746e-01,5.235982e-02,1.742426e+05,4.965559e+04,2.160102e+04,4.211294e+04,31375.959904,0.753995,1.668949,11.303190,57.411296,2.936318,12.805560,0.055604,0.087621,36.387197
std,3.857116e+05,NaN,8.707857e+03,8.703891e+03,2.613124e+02,7.003013e+04,NaN,1.110204e+01,8.791592e-01,9.386606e-01,21.918642,26.573014,5.470397e+00,6.031029e-01,2.234968e+04,1.199456e+01,1.470467e-01,21.378738,5.813323e+04,7.690692,7.752087e-02,1.121222e+04,1.574101e+05,1.208277,2.992520,0.992505,1.685077,24.895737,4.279953e+04,22.911809,1.574418,2.700078,5466.115805,20.905835,3.671216e+04,1.584782,2.824735,2.541732,3.183829e+00,1.627043e+04,1.529880e+04,2.825679e+01,1.098283e-01,8.120499e+02,5.222528e+01,9.446471e+01,1.633459e+01,8.728708e+0

In [7]:
train.dtypes

id                                       int64
member_id                              float64
loan_amnt                              float64
funded_amnt                            float64
term                                    object
int_rate                                object
installment                            float64
grade                                   object
sub_grade                               object
emp_title                               object
emp_length                              object
home_ownership                          object
annual_inc                             float64
url                                    float64
desc                                    object
purpose                                 object
title                                   object
zip_code                                object
addr_state                              object
dti                                    float64
delinq_2yrs                            float64
earliest_cr_l

In [8]:
train.isnull().sum()

id                                           0
member_id                              1309457
loan_amnt                                    0
funded_amnt                                  0
term                                         0
int_rate                                     0
installment                                  0
grade                                        0
sub_grade                                    0
emp_title                                83135
emp_length                               76101
home_ownership                               0
annual_inc                                   0
url                                    1309457
desc                                   1188346
purpose                                      0
title                                    16331
zip_code                                     1
addr_state                                   0
dti                                        327
delinq_2yrs                                  0
earliest_cr_l

In [9]:
for col in train.columns:
    if train[col].isnull().sum() >= 1000000:
        print(col)

member_id
url
desc
mths_since_last_record
annual_inc_joint
dti_joint
revol_bal_joint
sec_app_earliest_cr_line
sec_app_inq_last_6mths
sec_app_mort_acc
sec_app_open_acc
sec_app_revol_util
sec_app_open_act_il
sec_app_num_rev_accts
sec_app_chargeoff_within_12_mths
sec_app_collections_12_mths_ex_med
sec_app_mths_since_last_major_derog


In [10]:
train.shape

(1309457, 103)

In [11]:
train.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,1246716,NaN,22000.0,22000.0,36 months,5.32%,662.53,A,A1,Transition Coordinator,10+ years,MORTGAGE,126000.0,NaN,NaN,home_improvement,Home improvement,604xx,IL,28.99,0.0,Oct-2000,0.0,43.0,NaN,12.0,0.0,9371.0,51%,18.0,w,0.0,29.0,Individual,NaN,NaN,0.0,0.0,367985.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18351.0,NaN,NaN,NaN,2.0,30665.0,3250.0,69.0,0.0,0.0,145.0,182.0,5.0,2.0,2.0,57.0,NaN,18.0,NaN,2.0,3.0,7.0,4.0,4.0,7.0,7.0,9.0,7.0,12.0,NaN,0.0,0.0,2.0,89.0,0.0,0.0,0.0,451899.0,198749.0,9750.0,210298.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,1678744,NaN,10000.0,10000.0,36 months,5.32%,301.15,A,A1,Director HQ Controlling,4 years,MORTGAGE,120000.0,NaN,NaN,debt_consolidation,Debt consolidation,019xx,MA,10.69,1.0,Jan-2002,0.0,13.0,NaN,13.0,0.0,9145.0,35%,46.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,40945.0,0.0,9.0,0.0,4.0,13.0,31800.0,66.0,0.0,0.0,5548.0,55.0,26100.0,0.0,1.0,0.0,4.0,3150.0,16355.0,35.9,0.0,0.0,168.0,116.0,31.0,13.0,0.0,31.0,NaN,20.0,13.0,0.0,2.0,2.0,3.0,4.0,41.0,4.0,5.0,2.0,13.0,0.0,0.0,0.0,0.0,97.8,0.0,0.0,0.0,74267.0,40945.0,25500.0,48167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,1149018,NaN,7000.0,7000.0,36 months,6.97%,216.05,A,A3,NaN,NaN,RENT,80000.0,NaN,NaN,major_purchase,Major purchase,653xx,MO,29.39,0.0,Jan-2002,2.0,NaN,NaN,13.0,0.0,3703.0,10.4%,20.0,f,0.0,NaN,Individual,NaN,NaN,0.0,0.0,78269.0,1.0,5.0,2.0,3.0,7.0,74566.0,66.0,2.0,3.0,546.0,45.0,35500.0,2.0,0.0,5.0,6.0,6021.0,17930.0,6.1,0.0,0.0,170.0,44.0,4.0,4.0,0.0,4.0,NaN,1.0,NaN,0.0,3.0,6.0,4.0,5.0,10.0,8.0,10.0,6.0,13.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,131514.0,78269.0,19100.0,96014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,1349740,NaN,1400.0,1400.0,36 months,13.67%,47.63,C,C4,Administrative Assistant,5 years,RENT,49000.0,NaN,NaN,medical,Medical expenses,770xx,TX,25.07,0.0,Aug-2004,0.0,46.0,NaN,12.0,0.0,19979.0,79.6%,18.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,38644.0,1.0,1.0,1.0,1.0,5.0,18665.0,96.0,0.0,2.0,7612.0,87.0,25100.0,0.0,0.0,0.0,3.0,3220.0,1411.0,93.2,0.0,0.0,111.0,136.0,16.0,5.0,0.0,16.0,NaN,16.0,46.0,0.0,7.0,8.0,7.0,8.0,2.0,11.0,16.0,8.0,12.0,0.0,0.0,0.0,1.0,94.4,100.0,0.0,0.0,44615.0,38644.0,20800.0,19515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,1234365,NaN,19200.0,19200.0,36 months,8.18%,603.26,B,B1,Plant Electrical Engineer,< 1 year,RENT,150000.0,NaN,NaN,credit_card,Credit card refinancing,971xx,OR,16.18,0.0,Oct-1997,3.0,NaN,78.0,8.0,1.0,27000.0,56%,17.0,f,0.0,31

In [12]:
print(train['disbursement_method'].unique())

['Cash' 'DirectPay']


In [13]:
X_train = train
y_train = train_labels['charged_off']
X_test = test

In [14]:
def wrangle(X):
    
    X = X.copy()
    
    def remove_months(string):
        return int(string.strip(' months'))

    def remove_percent(string):
        if string == str:
            return float(string.strip(' %'))
        elif string == float:
            return float(string)
        elif string == int:
            return int(string)

    def remove_years(string):
        if "< 1 year" in string:
            return float(string.replace('< 1 year',0.5))
        elif "10+ years" in string:
            return float(string.replace('10+ years', 10.0))
        else:
            return float(string.strip(' years'))

    def remove_first4char(string):
        return int(string[4:])

    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns=['member_id', 'url', 'desc'])  # All null
    X = X.drop(columns='title')  # Duplicative of purpose
    X = X.drop(columns='grade')  # Duplicative of sub_grade
    
    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)

    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
        
    # Transform earliest_cr_line to an integer: how many days it's been open
    X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
    X['earliest_cr_line'] = pd.Timestamp.today() - X['earliest_cr_line']
    X['earliest_cr_line'] = X['earliest_cr_line'].dt.days
    
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)
    
    # Drop categoricals with high cardinality
    X = X.drop(columns=['emp_title', 'zip_code'])
    
    # Transform features with many nulls to binary flags
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

    for col in many_nulls:
        X[col] = X[col].isnull()
    
    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mean())
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(X_train)
X_test  = wrangle(X_test)
X_train.shape, X_test.shape

((1309457, 98), (26724, 98))

In [15]:
X_train['charged_off'] = y_train

In [16]:
X_train = X_train.sample(696969,random_state=42)
len(X_train)

696969

In [17]:
y_train = X_train['charged_off']

In [18]:
len(y_train)

696969

In [19]:
X_train.drop(columns=['charged_off'],inplace=True)

In [20]:
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier


pipe = make_pipeline(
    ce.OrdinalEncoder(), 
    DecisionTreeClassifier(max_depth=5, class_weight='balanced')
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc')

array([0.70002148, 0.69979996, 0.70294245, 0.69889998, 0.69988997])

In [21]:
from sklearn.ensemble import RandomForestClassifier

pipe = make_pipeline(
    ce.OrdinalEncoder(), 
    RandomForestClassifier(
        n_estimators=100, 
        class_weight='balanced', 
        min_samples_leaf=0.005, 
        oob_score=True, 
        n_jobs=-1)
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', verbose=10)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ........................ , score=0.713604944746056, total=  45.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   45.9s remaining:    0.0s


[CV] ....................... , score=0.7140554988602434, total=  46.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s


...

[CV] ....................... , score=0.7123817724240731, total=  43.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.0min remaining:    0.0s


[CV] ....................... , score=0.7145921948978968, total=  43.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.8min finished


array([0.71360494, 0.7140555 , 0.71747395, 0.71238177, 0.71459219])

In [22]:
from sklearn.metrics import roc_auc_score
%%time
pipe.fit(X_train, y_train)
y_pred_proba = pipe.named_steps['randomforestclassifier'].oob_decision_function_[:, 1]
print('ROC AUC, Out-of-Bag estimate:', roc_auc_score(y_train, y_pred_proba))



UsageError: Line magic function `%%time` not found.


In [23]:

max_depths = list(range(2, 12, 2)) + [None]

for max_depth in max_depths:
    
    pipe = make_pipeline(
        ce.OrdinalEncoder(), 
        RandomForestClassifier(
            n_estimators=100, 
            class_weight='balanced', 
            max_depth=max_depth, 
            oob_score=True, 
            n_jobs=-1
        )
    )
        
    pipe.fit(X_train, y_train)
    y_pred_proba = pipe.named_steps['randomforestclassifier'].oob_decision_function_[:, 1]
    print('Max Depth:', max_depth)
    print('ROC AUC, OOB:', roc_auc_score(y_train, y_pred_proba))
    
    

Max Depth: 2
ROC AUC, OOB: 0.6954342282483897
Max Depth: 4
ROC AUC, OOB: 0.7079807504125476
Max Depth: 6
ROC AUC, OOB: 0.7121813236512475
Max Depth: 8
ROC AUC, OOB: 0.7164003059893702
Max Depth: 10
ROC AUC, OOB: 0.7181671275289466
Max Depth: None
ROC AUC, OOB: 0.6969678642182601


In [24]:
# import matplotlib.pyplot as plt

# def show_feature_importances(
#     pipe, X, y, estimator_name='randomforestclassifier', 
#     n=20, figsize=(8, 8)):
    
#     # pipe must not change dimensions of X dataframe
#     pipe.fit(X, y)
    
#     importances = pd.Series(
#         pipe.named_steps[estimator_name].feature_importances_, 
#         X.columns)

#     top_n = importances.sort_values(ascending=False)[:n]
    
#     plt.figure(figsize=figsize)
#     top_n.sort_values().plot.barh(color='grey')

    
# show_feature_importances(pipe, X_train, y_train)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# cross_val_score(pipe, X_train.drop(columns='sub_grade'), y_train, cv=5, scoring='roc_auc')



In [ ]:
# show_feature_importances(pipe, X_train, y_train)

In [ ]:
# cross_val_score(pipe, X_train.drop(columns=['sub_grade', 'int_rate']), y_train, cv=5, scoring='roc_auc')



In [32]:
for col in X_train.columns:
    if col in X_test:
        pass
    else:
        print(col)

In [33]:
X_train.shape

(696969, 98)

In [ ]:
pipe = make_pipeline(
        ce.OrdinalEncoder(), 
        RandomForestClassifier(
            n_estimators=100, 
            class_weight='balanced', 
            max_depth=10, 
            n_jobs=-1,
            oob_score=True,
            verbose=5
        )
    )
pipe.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.8s


building tree 10 of 100
building tree 11 of 100
building tree 12 of 100building tree 13 of 100
building tree 14 of 100

building tree 15 of 100
building tree 16 of 100


In [ ]:
submission = sample_submission.copy()
submission['charged_off'] = pipe.predict(X_test)
submission.head()

In [ ]:
submission.to_csv('submission-04.csv',index=False)

In [21]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestClassifier

encoder = ce.OrdinalEncoder()
X_train_transformed = encoder.fit_transform(X_train)

model = RandomForestClassifier(
    n_estimators=100, 
    class_weight='balanced', 
    min_samples_leaf=0.005, 
    n_jobs=-1)

model.fit(X_train_transformed, y_train)
permuter = PermutationImportance(model, scoring='roc_auc', n_iter=1, cv='prefit')
permuter.fit(X_train_transformed, y_train)

PermutationImportance(cv='prefit',
           estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=0.005,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
           n_iter=1, random_state=None, refit=True, scoring='roc_auc')

In [22]:
eli5.show_weights(permuter, top=None, feature_names=X_train_transformed.columns.tolist())



Weight,Feature
0.0213 ± 0.0000,sub_grade
0.0178 ± 0.0000,int_rate
0.0129 ± 0.0000,term
0.0033 ± 0.0000,dti
0.0026 ± 0.0000,acc_open_past_24mths
0.0016 ± 0.0000,annual_inc
0.0013 ± 0.0000,tot_hi_cred_lim
0.0012 ± 0.0000,funded_amnt
0.0012 ± 0.0000,avg_cur_bal
0.0010 ± 0.0000,mort_acc


In [30]:
X_test_transformed = encoder.fit_transform(X_test)

In [31]:
submission = sample_submission.copy()
submission['charged_off'] = permuter.predict(X_test_transformed)
print(submission['charged_off'].value_counts())
submission.head()

ValueError: Number of features of the model must match the input. Model n_features is 73 and input n_features is 98 

In [27]:
subset = X_train_transformed.columns[permuter.feature_importances_ > 0]
cross_val_score(pipe, X_train[subset], y_train, cv=5, scoring='roc_auc', verbose=10)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=0.005,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [34]:
len(subset)

73

In [56]:
X_test_transformed[subset].shape

(26724, 73)

In [60]:
train = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def wrangle(X):
    
    X = X.copy()
    
    def remove_months(string):
        return int(string.strip(' months'))

    def remove_percent(string):
        if string == str:
            return float(string.strip(' %'))
        elif string == float:
            return float(string)
        elif string == int:
            return int(string)

    def remove_years(string):
        if "< 1 year" in string:
            return float(string.replace('< 1 year',0.5))
        elif "10+ years" in string:
            return float(string.replace('10+ years', 10.0))
        else:
            return float(string.strip(' years'))

    def remove_first4char(string):
        return int(string[4:])

    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns=['member_id', 'url', 'desc'])  # All null
    X = X.drop(columns='title')  # Duplicative of purpose
    X = X.drop(columns='grade')  # Duplicative of sub_grade
    
    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)

    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
        
    # Transform earliest_cr_line to an integer: how many days it's been open
    X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
    X['earliest_cr_line'] = pd.Timestamp.today() - X['earliest_cr_line']
    X['earliest_cr_line'] = X['earliest_cr_line'].dt.days
    
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)
    
    # Drop categoricals with high cardinality
    X = X.drop(columns=['emp_title', 'zip_code'])
    
    # Transform features with many nulls to binary flags
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

    for col in many_nulls:
        X[col] = X[col].isnull()
    
    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mean())
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(train)
X_test  = wrangle(test)
y_train = train_labels['charged_off']
X_train.shape, X_test.shape

((1309457, 98), (26724, 98))

In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier

pipe = make_pipeline(
        ce.OrdinalEncoder(), 
        RandomForestClassifier(
            n_estimators=100, 
            class_weight='balanced', 
            max_depth=10,
            min_samples_leaf=0.005,
            oob_score=True, 
            n_jobs=-1
        )
    )
# cross_val_score(pipe, X_train[subset], y_train, cv=5, scoring='roc_auc', verbose=10)


X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

encoder = ce.OrdinalEncoder()
X_train = encoder.fit_transform(X_train)
X_val = encoder.transform(X_val)
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_pred_proba = gb.predict_proba(X_val)[:,1]
print('Validation ROC AUC:', roc_auc_score(y_val, y_pred_proba))



Validation ROC AUC: 0.7242728852862521


In [70]:
X_val.shape

(261892, 98)

In [72]:
X_train.shape

(1047565, 98)

In [74]:
X_test = encoder.fit_transform(X_test)

In [75]:
X_test.shape

(26724, 98)

In [76]:
submission = sample_submission.copy()
submission['charged_off'] = gb.predict(X_test)
print(submission['charged_off'].value_counts())
submission.head()

0    26008
1      716
Name: charged_off, dtype: int64


,id,charged_off
0,2264704,0
1,1896343,0
2,2077525,0
3,1846827,0
4,1210251,0


In [80]:
test = pd.read_csv('test_features.csv')
test = wrangle(test)
X_test = encoder.transform(test)
X_test.head()

,loan_amnt,funded_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,emp_title_teacher,emp_title_manager,emp_title_owner
0,20000.0,20000.0,1,8.18,628.39,2.1,False,1,64000.00,2,3,12.45,0.0,5988,0.0,False,True,9.0,0.0,22607.0,82.2,12.0,2,0.0,True,1,True,True,0.0,50.000000,36978.000000,1.047456,2.789768,0.774582,1.731872,True,36042.528379,True,1.414579,3.010725,5615.552202,58.286474,27500.000000,1.082479,1.555429,2.278759,0.0,4109.000000,False,False,0.0,0.0,False,147.000000,56.000000,31.000000,1.0,False,False,True,False,0.000000,3.000000,3.000000,4.000000,6.000000,4.000000,5.000000,7.000000,3.000000,9.000000,False,0.000000,0.000000,0.00000,91.70000,False,0.0,0.0,44000.000000,36978.0,25500.0,16500.000000,True,True,True,True,True,True,True,True,True,True,True,1,False,False,False
1,4000.0,4000.0,1,13.99,136.70,3.1,False,3,32000.00,2,11,16.20,0.0,4618,0.0,False,True,8.0,0.0,5240.0,77.1,12.0,2,0.0,True,1,True,True,0.0,220.028111,142207.041832,1.047456,2.789768,0.774582,1.731872,True,36042.528379,True,1.414579,3.010725,5615.552202,58.286474,33052.347985,1.082479,1.555429,2.278759,3.0,13559.537134,False,False,0.0,0.0,True,181.961352,13.051936,7.830077,0.0,False,True,False,False,0.504541,3.669589,5.690898,4.771956,8.149754,8.515903,8.328602,14.663063,5.638962,11.702314,True,0.002831,0.088697,2.19654,94.14558,False,0.0,0.0,175817.145941,15701.0,6500.0,41952.408807,True,True,True,True,True,True,True,True,True,True,True,1,False,False,False
2,10000.0,10000.0,1,10.99,327.34,2.2,False,1,89000.00,2,17,18.04,2.0,4922,0.0,False,True,12.0,0.0,10675.0,87.6,21.0,1,0.0,False,1,True,True,0.0,0.000000,42555.000000,1.047456,2.789768,0.774582,1.731872,True,36042.528379,True,1.414579,3.010725,5615.552202,58.286474,12500.000000,1.082479,1.555429,2.278759,3.0,3869.000000,False,False,0.0,0.0,False,95.000000,8.000000,8.000000,0.0,False,False,True,False,1.000000,3.000000,8.000000,3.000000,3.000000,9.000000,9.000000,11.000000,8.000000,12.000000,False,0.000000,2.000000,1.00000,100.00000,False,0.0,0.0,58690.000000,42555.0,1400.0,46190.000000,True,True,True,True,True,True,True,True,True,True,True,1,False,False,False
3,12000.0,12000.0,2,14.31,281.16,3.4,False,3,35360.00,2,28,30.96,0.0,5837,0.0,False,True,7.0,0.0,6444.0,37.9,16.0,1,0.0,True,1,True,True,0.0,0.000000,86687.000000,1.047456,2.789768,0.774582,1.731872,True,36042.528379,True,1.414579,3.010725,5615.552202,58.286474,17000.000000,1.082479,1.555429,2.278759,2.0,12384.000000,False,False,0.0,0.0,False,140.000000,9.000000,9.000000,3.0,False,False,True,False,0.000000,3.000000,4.000000,4.000000,6.000000,

In [81]:
submission = sample_submission.copy()
submission['charged_off'] = gb.predict(X_test)
print(submission['charged_off'].value_counts())
submission.head()

0    26008
1      716
Name: charged_off, dtype: int64


,id,charged_off
0,2264704,0
1,1896343,0
2,2077525,0
3,1846827,0
4,1210251,0


In [ ]:
gb.fit(X_train, y_train)

In [82]:
import numpy as np
y_pred = (y_pred_proba >= 0.5).astype(int)
confidence = np.abs(y_pred_proba - 0.5)
preds = pd.DataFrame({'y_val': y_val, 'y_pred': y_pred, 
                      'y_pred_proba': y_pred_proba, 
                      'confidence': confidence})

preds.head()

,y_val,y_pred,y_pred_proba,confidence
669688,0,0,0.132263,0.367737
990047,0,0,0.123992,0.376008
410168,0,0,0.272568,0.227432
356197,0,0,0.414733,0.085267
414684,0,0,0.096974,0.403026


In [83]:

# True positives, with high confidence
preds[(y_val==1) & (y_pred==1)].sort_values(by='confidence', ascending=False).head()

,y_val,y_pred,y_pred_proba,confidence
1299821,1,1,0.746530,0.246530
1164550,1,1,0.740245,0.240245
255793,1,1,0.740191,0.240191
756400,1,1,0.736516,0.236516
154964,1,1,0.732059,0.232059


In [84]:
preds[(y_val==0) & (y_pred==1)].sort_values(by='confidence', ascending=False).head()

,y_val,y_pred,y_pred_proba,confidence
1091986,0,1,0.723191,0.223191
624208,0,1,0.722098,0.222098
172030,0,1,0.714562,0.214562
741489,0,1,0.712676,0.212676
1269649,0,1,0.709826,0.209826


In [87]:
data_for_pred = X_val.loc[1091986]
data_for_pred

loan_amnt                               30000
funded_amnt                             30000
term                                        2
int_rate                                 26.3
installment                            903.55
sub_grade                                 5.5
emp_length                               True
home_ownership                              3
annual_inc                               1000
purpose                                     2
addr_state                                  7
dti                                    324.73
delinq_2yrs                                 0
earliest_cr_line                         1727
inq_last_6mths                              2
mths_since_last_delinq                   True
mths_since_last_record                   True
open_acc                                    6
pub_rec                                     0
revol_bal                                8694
revol_util                               57.2
total_acc                         

In [88]:
preds[(y_val==1) & (y_pred==0)].sort_values(by='confidence', ascending=False).head()


,y_val,y_pred,y_pred_proba,confidence
826914,1,0,0.029805,0.470195
471177,1,0,0.030979,0.469021
1103462,1,0,0.032175,0.467825
851905,1,0,0.032298,0.467702
1123605,1,0,0.032490,0.467510


In [89]:
preds.sort_values(by='confidence', ascending=True).head()


,y_val,y_pred,y_pred_proba,confidence
554871,0,0,0.499984,0.000016
1241784,1,0,0.499981,0.000019
1090194,0,0,0.499962,0.000038
1140303,1,0,0.499954,0.000046
1123448,1,0,0.499954,0.000046


In [91]:
len(y_pred)

261892

In [94]:
len(submission)

26724

In [53]:
pipe.fit(X_train[subset],y_train)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['term', 'home_ownership', 'purpose', 'addr_state', 'initial_list_status'],
        drop_invariant=False, handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'term', 'mapping': [(' 36 months', 1), (' 60 months', 2)]}, {'col': 'home_own...mators=100, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False))])

In [55]:
submission = sample_submission.copy()
submission['charged_off'] = pipe.predict(X_test[subset])
print(submission['charged_off'].value_counts())
submission.head()

0    14804
1    11920
Name: charged_off, dtype: int64


,id,charged_off
0,2264704,0
1,1896343,0
2,2077525,0
3,1846827,1
4,1210251,0


In [47]:
submission = sample_submission.copy()
submission['charged_off'] = pipe.predict(X_test[subset])
print(submission['charged_off'].value_counts())
submission.head()

0    15487
1    11237
Name: charged_off, dtype: int64


,id,charged_off
0,2264704,0
1,1896343,0
2,2077525,0
3,1846827,1
4,1210251,0


In [48]:
submission.to_csv('submission-06.csv',index=False)

In [95]:
train = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def wrangle(X):
    
    X = X.copy()
    
    def remove_months(string):
        return int(string.strip(' months'))

    def remove_percent(string):
        if string == str:
            return float(string.strip(' %'))
        elif string == float:
            return float(string)
        elif string == int:
            return int(string)

    def remove_years(string):
        if "< 1 year" in string:
            return float(string.replace('< 1 year',0.5))
        elif "10+ years" in string:
            return float(string.replace('10+ years', 10.0))
        else:
            return float(string.strip(' years'))

    def remove_first4char(string):
        return int(string[4:])

    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns=['member_id', 'url', 'desc'])  # All null
    X = X.drop(columns='title')  # Duplicative of purpose
    X = X.drop(columns='grade')  # Duplicative of sub_grade
    
    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)

    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
        
    # Transform earliest_cr_line to an integer: how many days it's been open
    X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
    X['earliest_cr_line'] = pd.Timestamp.today() - X['earliest_cr_line']
    X['earliest_cr_line'] = X['earliest_cr_line'].dt.days
    
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)
    
    # Drop categoricals with high cardinality
    X = X.drop(columns=['emp_title', 'zip_code'])
    
    # Transform features with many nulls to binary flags
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

    for col in many_nulls:
        X[col] = X[col].isnull()
    
    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mean())
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(train)
X_test  = wrangle(test)
y_train = train_labels['charged_off']
X_train.shape, X_test.shape

((1309457, 98), (26724, 98))

In [98]:
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

def make_predictions(pipeline, X_train, y_train, X_test, string):
    pipeline.fit(X_train, y_train)
    sample_submission = pd.read_csv('sample_submission.csv') 
    submission = sample_submission.copy() 
    submission['charged_off'] = pipeline.predict_proba(X_test)[:, 1] 
    submission.to_csv(string, index=False)

    
pipe = make_pipeline(
    ce.BinaryEncoder(), 
    XGBClassifier(
        learning_rate = 0.1, 
        min_child_weight = 5,
        max_depth = 4,
        gamma=0.2,
        max_delta_step = 0,
        subsample = 1,
        colsample_bytree = 0.4,
        colsample_bylevel = 1,
        n_estimators=200,
        alpha = 0,
        scale_pos_weight = 1,
        eval_metric = "auc",
        xgbclassifier__silent = False,
        n_jobs=-1)
)

make_predictions(pipe, X_train, y_train, X_test, 'submission-08.csv')